# Sentiment classification - close to the state of the art

The task of classifying sentiments of texts (for example movie or product reviews) has high practical significance in online marketing as well as financial prediction. This is a non-trivial task, since the concept of sentiment is not easily captured.

For this assignment you have to use the larger [IMDB sentiment](https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) benchmark dataset from Stanford, an achieve close to state of the art results.

The task is to try out multiple models in ascending complexity, namely:

1. TFIDF + classical statistical model (eg. RandomForest)
2. LSTM classification model
3. LSTM model, where the embeddings are initialized with pre-trained word vectors
4. fastText model
5. BERT based model (you are advised to use a pre-trained one and finetune, since the resource consumption is considerable!)

You should get over 90% validation accuracy (though nearly 94 is achievable).

You are allowed to use any library or tool, though the Keras environment, and some wrappers on top (ie. Ktrain) make your life easier.





__Groups__
This assignment is to be completed individually, two weeks after the class has finished. For the precise deadline please see canvas.

__Format of submission__
You need to submit a pdf of your Google Collab notebooks.

__Due date__
Two weeks after the class has finished. For the precise deadline please see canvas.

Grade distribution:
1. TFIDF + classical statistical model (eg. RandomForest) (25% of the final grade)
2. LSTM classification model (15% of the final grade)
3. LSTM model, where the embeddings are initialized with pre-trained word vectors, e.g. fastText, GloVe etc. (15% of the final grade)
4. fastText model (15% of the final grade)
5. BERT based model (you are advised to use a pre-trained one and finetune it, since the resource consumption is considerable!) (30% of the final grade). For BERT you should get over 90% validation accuracy (though nearly 94% is achievable).


__For each of the models, the marks will be awarded according to the following three criteria__:

(1) The (appropriately measured) accuracy of your prediction for the task. The more accurate the prediction is, the better. Note that you need to validate the predictive accuracy of your model on a hold-out of unseen data that the model has not been trained with.

(2) How well you motivate the use of the model - what in this model's structure makes it suited for representing sentiment? After using the model for the task how well you evaluate the accuracy you got for each model and discuss the main advantages and disadvantages the model has in the particular modelling task. At best you take part of the modelling to support your arguments.

(3) The consistency of your take-aways, i.e. what you have learned from your analyses. Also, analyze when the model is good and when and where it does not predict well.

Please make sure that you comment with # on the separates steps of the code you have produced. For the verbal description and analyses plesae insert markdown cells.


__Plagiarism__: The Frankfurt School does not accept any plagiarism. Data science is a collaborative exercise and you can discuss the research question with your classmates from other groups, if you like. You must not copy any code or text though. Plagiarism will be prosecuted and will result in a mark of 0 and you failing this class.

After carefully reading this document and having had a look at the data you may still have questions. Please submit those question to the public Q&A board in canvas and we will answer each question, so 

# Data download

In [19]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz
!ls

--2020-12-02 20:22:16--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.2’

aclImdb_v1.tar.gz.2 100%[===================>]  80.23M  22.1MB/s    in 5.4s    

2020-12-02 20:22:21 (14.9 MB/s) - ‘aclImdb_v1.tar.gz.2’ saved [84125825/84125825]

aclImdb		   aclImdb_v1.tar.gz.1	drive	   sample_data
aclImdb_v1.tar.gz  aclImdb_v1.tar.gz.2	IMDB_Data


# Alternative with tf.datasets

In [20]:
!pip install tensorflow-datasets > /dev/null

In [21]:
import tensorflow_datasets as tfds
import pandas as pd
import os

In [22]:
(ds_train,ds_test),ds_info = tfds.load(
    name="imdb_reviews",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True   
)

# Storing the files

1. Out of all the files created we make the data ready in the form of .pkl files keeping in mind we work in Colaboratory.

2. However you can specify the directory (dir) in your specified file

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Go to the file created in Colab and specify your path
dir = '/content'

The command will try to make a directory in your personal drive to store all files and embedding ".txt" files which will be used to train in Deep Learning

In [25]:
# Make directory "IMDB" to store all types of files
try:
   os.mkdir(dir + "/IMDB_Data")
except:
    print("File exists in your personal drive")

File exists in your personal drive


In [26]:
# Go to the file created in Colab
dir = "/content/IMDB_Data"

<div class="alert alert-block alert-danger">
<b>Ratio:</b> Data is strictly prepared on test and validation split of 50 %
</div>

Train data is stored in the form of .pkl files in data directory

In [27]:
df = pd.DataFrame()
# Making a dataframe and storing the "text" and "label" of train
for i in ds_train.as_numpy_iterator():
      new_row = {'Text':i[0], 'Label':int(i[1])}
      df = df.append(new_row, ignore_index=True)

In [28]:
df.head(5)
print(df.shape)

(25000, 2)


In [29]:
df = df.to_pickle(dir + "/train.pkl")

Test data is stored in the form of .pkl files in test directory

In [30]:
df = pd.DataFrame()
# Making a dataframe and storing the "text" and "label" of test
for i in ds_test.as_numpy_iterator():
      new_row = {'Text':i[0], 'Label':int(i[1])}
      df = df.append(new_row, ignore_index=True)

In [31]:
df = df.to_pickle(dir + "/test.pkl")

Store the data in personal drive folder to make sure you can access the data in multiple Colab notebooks

In [32]:
%cd /content/drive/MyDrive/
%cp -av /content/IMDB_Data IMDB_Data

/content/drive/MyDrive
'/content/IMDB_Data' -> 'IMDB_Data/IMDB_Data'
'/content/IMDB_Data/train.pkl' -> 'IMDB_Data/IMDB_Data/train.pkl'
'/content/IMDB_Data/test.pkl' -> 'IMDB_Data/IMDB_Data/test.pkl'


Let's go now to this [Notebook](https://colab.research.google.com/drive/13DGUFAMrwbvbsYJdp5cN0eMkaAuGkZIY?usp=sharing) to prepare the data